In [7]:
import pandas as pd
train_df = pd.read_csv('/Users/geumjong-yeon/ML_Guide/Perfect_ML/data/ratings_train.txt',
                      sep = '\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [8]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [9]:
 train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [11]:
import re

train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경 (정규 표현식으로 \d 는 숫자를 의미함.)
train_df.drop('id', axis=1, inplace=True)
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+',' ',x))

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('/Users/geumjong-yeon/ML_Guide/Perfect_ML/data/ratings_test.txt',
                      sep = '\t')
test_df = test_df.fillna(' ')
test_df.drop('id', axis=1, inplace=True)
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r'\d+', ' ', x))

In [12]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 Text를 형태소 단어로 토큰화 하여 list 객체 변환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

tw_tokenizer('첫째')

/opt/anaconda3/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['첫째']

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3,
                            max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [16]:
lg_clf = LogisticRegression(random_state=0)
params = {'C' : [1, 3.5, 4.5, 5.5, 10]}

# Parameter C 최적화를 위해 GridSearchCV를 화용
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', random_state=0)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params, round(grid_cv.best_score_,4))

TypeError: __init__() got an unexpected keyword argument 'random_state'

In [ ]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test= tfidf_vect.transform(test_df['document'])

# Classifer는 gridserachCV에서 최적파라미터로 학습된 classifier를 그대로이용.
best_estimator = grid_Cv.best_estimator_
preds=best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도:', accuracy_score(test_df['label'], preds))

In [ ]:
I'm 